# Reading and storing data utility functions

In [1]:
import json

filename = 'top_CS_researcher_by_h_index.json'

def read_data(filepath=filename):
    try:
        with open(filepath, 'r') as fp:
            return json.load(fp)
    except FileNotFoundError:
        return []

def store_data(data, filepath=filename):
    with open(filepath, 'w') as fp:
        return json.dump(data, fp)

# Fetch Publications for an author

* We find the top authors in Computer Science - based on `h-index`
* We find the `top 100 pubs` for every author and then
* We calculate
    - h-index
    - c-score (log(total_citations) + log(h) + log(hm) + log(%first_author) + log(%first_last) + log(%first_last_single)
    - h-leadership-index (inverted bell curve)

* Things to remember:
    - we are not ranking individuals but creating an index
* Other notes
    - cscore does not include Field-wise citation index

In [2]:
import pycountry
import pandas as pd
import requests
import xmltodict

# Read the Stanford 2% ranking excel file
df = pd.read_excel('Table_1_Authors_career_2022_pubs_since_1788_wopp_extracted_202310.xlsx', sheet_name='Data', engine='openpyxl')
# We are only considering the Computer Science subject for this study
df = df.loc[df['sm-field'] == 'Information & Communication Technologies']
# NOTE: The cutoff year is 2022,and h column name changes based on cutoff year
df = df.sort_values(by='h22', ascending=False)

# A mapping between Stanford's top 2% ranking `sm-field` and Scopus API `SUBJECTAREA`
subject_areas_mapping = {
    'Information & Communication Technologies': ['COMP', 'MULT']
}

# Set up your Scopus API key
api_key = '5aa908d24ec7e71ef0cf68cb3bff134d'

# Define the Scopus API endpoint for author search
scopus_search_url = 'https://api.elsevier.com/content/search/author'

# Define the Scopus API endpoint for retrieving author's publications
scopus_search_publications_url = 'https://api.elsevier.com/content/search/scopus'

# Define the Scopus Abstract Retrieval API to get detailed information about the publication
scopus_abstract_url = f'https://api.elsevier.com/content/abstract/eid'

# Set up headers with your API key
headers = {
    'X-ELS-APIKey': api_key,
}


# Utility methods
def get_author_names(author_full_name):
    # Split the full name into first and last names
    names = author_full_name.split(', ', 1)
    first_name = names[-1]
    last_name = names[0] if len(names)==2 else ''
    return first_name, last_name

def get_country_name(country_code):
    try:
        country_name = pycountry.countries.get(alpha_3=country_code).name
        return country_name
    except AttributeError:
        # Handle cases where the country code is not found
        return ''

In [3]:
# Function to search for an author in Scopus using name, affiliation, country, and field
def search_author(author_name, affiliation, country_code, field):
    first_name, last_name = get_author_names(author_name)
    subject_areas = subject_areas_mapping.get(field, [])
    country_name = get_country_name(country_code)

    query = f'AUTHLASTNAME({last_name}) AND AUTHFIRST({first_name}) AND {" OR ".join(map(lambda s: f"SUBJAREA({s})", subject_areas))}'
    response = requests.get(scopus_search_url, params={'query': query, 'count': 200}, headers=headers)

    if response.status_code == 200:
        results = response.json().get('search-results', {}).get('entry', [])
        results.sort(key=lambda x: x.get('document-count', 0), reverse=True)
        results.sort(key=lambda x: x.get('affiliation-current', {}).get('affiliation-country', '') == country_name, reverse=True)
        # Affiliation is excluded as it can change
        # results.sort(key=lambda x: x.get('affiliation-current', {}).get('affiliation-name', '') == affiliation, reverse=True)
        return results[0] if results else None
    else:
        return None

In [4]:
# Function to fetch all publications for an author using Scopus Author ID
def fetch_author_publications(author_id, publications=None, start_index=0, limit=100):
    if not publications:
        publications = []

    query = f'AU-ID({author_id})'
    response = requests.get(scopus_search_publications_url, params={
        'query': query,
        'start': start_index,
        'count': min(limit, 200), # Maximum can be 200
        'sort': 'citedby-count'
    }, headers=headers)

    if response.status_code == 200:
        search_results = response.json().get('search-results', {}).get('entry', [])

        for entry in search_results:
            authors = []
            abstract_url = f'{scopus_abstract_url}/{entry.get("eid", "")}'
            response_abstract = requests.get(abstract_url, headers=headers)
            if response_abstract.status_code == 200:
                author_data = xmltodict.parse(response_abstract.text).get('abstracts-retrieval-response', {}).get('authors', []).get('author', [])
                if not isinstance(author_data, list): # If only one author
                    author_data = [author_data]
                authors = map(lambda a: {
                        'scopus_id': a['@auid'],
                        'name': a.get('ce:indexed-name', ''),
                    }, author_data)

            publication_data = {
                'title': entry.get('dc:title', ''),
                'eid': entry.get('eid', ''),
                'citations': entry.get('citedby-count', 0),
                'authors': list(authors),
                'publication_name': entry.get('prism:publicationName', ''),
                'issn': entry.get('prism:issn', ''),
                'cover_date': entry.get('prism:coverDate', ''),
                'venue': entry.get('prism:aggregationType', ''),
                'volume': entry.get('prism:volume', ''),
                'issue': entry.get('prism:issueIdentifier', ''),
                'page_range': entry.get('prism:pageRange', ''),
                'doi': entry.get('prism:doi', ''),
            }
            publications.append(publication_data)

        # Fetch the next set of publications if available and limit is not reached
        start_index += len(search_results)
        if limit > start_index and start_index < int(response.json().get('search-results', {}).get('opensearch:totalResults', 0)):
            return fetch_author_publications(author_id, publications, start_index)

    return publications

In [ ]:
# main code
scopus_results=read_data()
def fetch_authors(stop_at=300):
    # Iterate through the rows of the DataFrame
    for index, (row_index, row) in enumerate(df.iterrows()):
        if index < len(scopus_results):
            # Since data was previously obtained for these authors, we can skip them
            print(f'{row["authfull"]}: Skipped')
            continue

        try:
            author_name = row['authfull']
            author_cscore = row['c']
            affiliation = row['inst_name']
            country_code = row['cntry']
            field = row['sm-field']
            print(author_name)

            # Search for the author in Scopus
            author_search_result = search_author(author_name, affiliation, country_code, field)

            # Process the search result as needed
            if author_search_result is not None:
                author_id = author_search_result.get('dc:identifier', '').split(':')[-1]
                author_publications = fetch_author_publications(author_id)
                scopus_results.append({
                    'scopus_id': author_id,
                    'name': author_name,
                    # 'cscore': author_cscore,
                    'publications': author_publications
                })
                if index == stop_at:
                    break
            else:
                raise Exception(f"Author not found for: {author_name}, Affiliation: {affiliation}, Country: {country_code}, Field: {field}")
        except Exception as e:
            print(e)

    store_data(scopus_results)
    # return scopus_results

fetch_authors()
# TODO: Find correlation between hl-index, h-index and c-score
# Find the remaining usage-limit for Scopus API

In [4]:
len(scopus_results)

301

# Calculate Metrics for the Author

In [2]:
import pandas as pd
from calculate import *
from IPython.display import display
import json

filename = 'top_CS_researcher_by_h_index.json'

def read_data(filepath=filename):
    try:
        with open(filepath, 'r') as fp:
            return json.load(fp)
    except FileNotFoundError:
        return []

rows = []
authors = read_data()
for author in authors:
    if h_index(author['publications']) < 50:
        # All top 300 authors have higher h-indices so
        # incorrect author got mined
        continue
    try:
        rows.append({
            'Name': author['name'],
            'Publications': len(author['publications']),
            'Total citations': total_citations(author['publications']),
            'Median citations': median_citations(author['publications']),
            'h-index': h_index(author['publications']),
            'h-frac-index': h_frac_index(author['publications']),
            'hm-index': hm_index(author['publications']),
            'h-leadership-index': h_leadership_index(author['scopus_id'], author['publications']),
            '% first author': percent_first_author(author['scopus_id'], author['publications']),
            '% last author': percent_last_author(author['scopus_id'], author['publications']),
            '% single author': percent_single_author(author['publications']),
            'Median author position': median_author_position(author['scopus_id'], author['publications']),
            # 'cscore':
            'i10-index': i10_index(author['publications']),
            'Average number of Authors': mean_coauthors(author['publications']),
            'Median number of Authors': median_coauthors(author['publications']),
        })
    except Exception as e:
        print(f"Error processing author: {author['name']}")

authors_df = pd.DataFrame(rows)
display(authors_df)

/home/hardik/Documents/metrics/.venv/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/hardik/Documents/metrics/.venv/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


,Name,Publications,Total citations,Median citations,h-index,h-frac-index,hm-index,h-leadership-index,% first author,% last author,% single author,Median author position,i10-index,Average number of Authors,Median number of Authors
0,"Jain, Anil",100,79918,407.0,100,47,0,100,30.0,55.0,2.0,2.0,100,3.24,3.0
1,"Bengio, Yoshua",100,291046,670.5,100,39,0,100,18.0,62.0,4.0,3.0,100,5.44,4.0
2,"Poor, H. Vincent",100,47812,315.5,100,48,0,100,4.0,75.0,0.0,4.0,100,4.62,4.0
3,"Han, Jiawei",100,58511,317.0,100,59,0,100,10.0,48.0,0.0,3.0,100,4.01,4.0
4,"Gool, Luc Van",100,85750,345.0,100,49,0,100,1.0,80.0,0.0,4.0,100,7.72,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,"Dongarra, Jack",100,16839,98.5,72,0,0,0,0.0,0.0,0.0,NaN,100,NaN,NaN
205,"Zeadally, Sherali",100,17809,134.0,80,0,0,0,0.0,0.0,0.0,NaN,100,NaN,NaN
206,"Blei, David M.",100,62987,115.0,75,0,0,0,0.0,0.0,0.0,NaN,100,NaN,NaN
207,"Sebe, Nicu",100,16949,116.0,79,0,0,0,0.0,0.0,0.0,NaN,100,NaN,NaN


## Update (bug observed)
* I have observed that for some authors, when mining their co-authors, Scopus API returns an empty list.
* This results in an overall incorrect calculation of the h-leadership index
* I am yet to trace down the root cause behind this and would work on this post my midsemester break.